PRIMERO PREDIGO EL MEJOR CORTE CON LA VALIDACIÓN

In [28]:
require( "data.table" )
# Carga de predicciones de probabilidad (202104)
p0 <- fread("/content/prediccion_202104_ensemble_4920.txt")[, .(numero_de_cliente, foto_mes, prob0 = prob)]
p1 <- fread("/content/prediccion_202104_ensemble_4921.txt")[, .(numero_de_cliente, foto_mes, prob1 = prob)]
p2 <- fread("/content/prediccion_202104_ensemble_4922.txt")[, .(numero_de_cliente, foto_mes, prob2 = prob)]
p3 <- fread("/content/prediccion_202104_ensemble_4923.txt")[, .(numero_de_cliente, foto_mes, prob3 = prob)]
p4 <- fread("/content/prediccion_202104_ensemble_4924.txt")[, .(numero_de_cliente, foto_mes, prob4 = prob)]

# Carga de tablas de ganancia (curvas de ganancia)
g0 <- fread("/content/ganancia_ensamble_4920.csv")
g1 <- fread("/content/ganancia_ensamble_4921.csv")
g2 <- fread("/content/ganancia_ensamble_4922.csv")
g3 <- fread("/content/ganancia_ensamble_4923.csv")
g4 <- fread("/content/ganancia_ensamble_4924.csv")

# 1. Encontrar la ganancia máxima para cada modelo
g0_max <- g0[which.max(ganancia_total)]$ganancia_total
g1_max <- g1[which.max(ganancia_total)]$ganancia_total
g2_max <- g2[which.max(ganancia_total)]$ganancia_total
g3_max <- g3[which.max(ganancia_total)]$ganancia_total
g4_max <- g4[which.max(ganancia_total)]$ganancia_total

# Definir los modelos A ENSAMBLAR y sus ganancias
ganancias_a_ensamblar <- c(g0_max, g3_max, g4_max)
# ganancias_a_ensamblar <- c(g0_max, g1_max, g2_max, g3_max, g4_max)

# Calcular la suma total de las ganancias de los modelos seleccionados
suma_total_ganancia <- sum(ganancias_a_ensamblar)

# 4. Calcular los pesos de ponderación
w0 <- g0_max / suma_total_ganancia
# w1 <- g1_max / suma_total_ganancia
# w2 <- g2_max / suma_total_ganancia
w3 <- g3_max / suma_total_ganancia
w4 <- g4_max / suma_total_ganancia

cat("--- Pesos de Ponderación (Basados en Ganancia Máxima 202104) ---\n")
cat(sprintf("Peso M4920 (w0): %.4f\n", w0))
# cat(sprintf("Peso M4921 (w1): %.4f\n", w1))
# cat(sprintf("Peso M4922 (w2): %.4f\n", w2))
cat(sprintf("Peso M4923 (w3): %.4f\n", w3))
cat(sprintf("Peso M4924 (w4): %.4f\n", w4))
cat(sprintf("Suma de Pesos: %.4f\n", w0 + w3 + w4))
# cat(sprintf("Suma de Pesos: %.4f\n", w0 + w1 + w2 + w3 + w4))
cat("----------------------------------------------------------------\n")


# Lista de Data.Tables a ensamblar
tablas_a_ensamblar <- list(p0, p3, p4)

# Combinar las probabilidades
ensamble <- Reduce(function(x, y) merge(x, y, by=c("numero_de_cliente","foto_mes")),
                   tablas_a_ensamblar)

# Calcular la probabilidad ponderada (Weighted Ensemble)
ensamble[, prob_ponderada := (w0 * prob0) + (w3 * prob3) + (w4 * prob4)]
# ensamble[, prob_ponderada := (w0 * prob0) + (w1 * prob1) + (w2 * prob2) + (w3 * prob3) + (w4 * prob4)]

# Preparar el resultado final (ranking)
prediccion_202104_ensemble_ponderado <- ensamble[, .(numero_de_cliente, foto_mes, prob = prob_ponderada)]

cat("Predicción ensemble ponderada (202104) calculada con éxito.\n")

--- Pesos de Ponderación (Basados en Ganancia Máxima 202104) ---
Peso M4920 (w0): 0.3369
Peso M4923 (w3): 0.3318
Peso M4924 (w4): 0.3314
Suma de Pesos: 1.0000
----------------------------------------------------------------
Predicción ensemble ponderada (202104) calculada con éxito.


In [32]:
require( "data.table" )
# leo el dataset
dataset <- fread("/content/competencia_01_crudo.csv")

# calculo el periodo0 consecutivo
dsimple <- dataset[, list(
    "pos" = .I,
    numero_de_cliente,
    periodo0 = as.integer(foto_mes/100)*12 +  foto_mes%%100 ) ]


# ordeno
setorder( dsimple, numero_de_cliente, periodo0 )

# calculo topes
periodo_ultimo <- dsimple[, max(periodo0) ]
periodo_anteultimo <- periodo_ultimo - 1


# calculo los leads de orden 1 y 2
dsimple[, c("periodo1", "periodo2") :=
    shift(periodo0, n=1:2, fill=NA, type="lead"),  numero_de_cliente ]

# assign most common class values = "CONTINUA"
dsimple[ periodo0 < periodo_anteultimo, clase_ternaria := "CONTINUA" ]

# calculo BAJA+1
dsimple[ periodo0 < periodo_ultimo &
    ( is.na(periodo1) | periodo0 + 1 < periodo1 ),
    clase_ternaria := "BAJA+1" ]

# calculo BAJA+2
dsimple[ periodo0 < periodo_anteultimo & (periodo0+1 == periodo1 )
    & ( is.na(periodo2) | periodo0 + 2 < periodo2 ),
    clase_ternaria := "BAJA+2" ]


# pego el resultado en el dataset original y grabo
setorder( dsimple, pos )
dataset[, clase_ternaria := dsimple$clase_ternaria ]

setorder( dataset, foto_mes, clase_ternaria, numero_de_cliente)
dataset[, .N, list(foto_mes, clase_ternaria)]

foto_mes,clase_ternaria,N
<int>,<chr>,<int>
202101,BAJA+1,622
202101,BAJA+2,825
202101,CONTINUA,160080
202102,BAJA+1,831
202102,BAJA+2,1032
202102,CONTINUA,160292
202103,BAJA+1,1039
202103,BAJA+2,951
202103,CONTINUA,161119


In [4]:
realidad_evaluar <- function( tb_ranking ) {

  # Agrupar para calcular las cantidades (qty) solo de los envíos (Predicted = 1L)
  tbl <- tb_ranking[Predicted==1L, list("qty"=.N), list(clase_ternaria)]

  res <- list()
  res$total <- tbl[, sum(qty * ifelse(clase_ternaria=="BAJA+2", 780000, -20000))]

  return( res )
}

In [29]:
require( "data.table" )


PARAM <- list(
    future = 202104,
    cortes = seq(6000, 13000, by= 100) # Tu rango de envíos
)

# Preparar el Data.table de Realidad (drealidad)
drealidad <- dataset[foto_mes == PARAM$future,
                     list(numero_de_cliente, foto_mes, clase_ternaria)]

# Crear la columna 'fold' para la división Público/Privado
# solo evalúo la ganancia total.
drealidad[, ultimo_digito := numero_de_cliente %% 10 ]
drealidad[, fold := 1L ]
drealidad[ultimo_digito >= 7, fold := 2L ]
drealidad[, ultimo_digito := NULL ] # Limpiar columna temporal

# Preparar el Data.table de Predicción (Ranking)
# Forzar tipo de dato a Integer para asegurar el merge
drealidad[, numero_de_cliente := as.integer(numero_de_cliente)]
prediccion_202104_ensemble_ponderado[, numero_de_cliente := as.integer(numero_de_cliente)]
tb_prediccion_ensemble <- prediccion_202104_ensemble_ponderado[, list(numero_de_cliente, prob)]

# 4. Unir el ranking de probabilidad (ensemble) con la realidad y el fold
drealidad_eval <- merge(drealidad,
                        tb_prediccion_ensemble,
                        by = "numero_de_cliente",
                        all.x = FALSE) # Usamos INNER JOIN para evitar clientes sin predicción

# Ordenar el ranking una sola vez
setorder(drealidad_eval, -prob)

# Inicializar la tabla para guardar la curva de ganancia
ganancias_ensemble_val <- data.table(
    envios = integer(),
    ganancia_total = numeric()
)


cat("\n--- Curva de Ganancia del Super-Ensemble (202104) ---\n")

for (envios in PARAM$cortes) {

    # Marcar la predicción binaria para el corte actual
    drealidad_eval[, Predicted := 0L] # Reseteamos
    drealidad_eval[1:envios, Predicted := 1L] # Marcamos el corte

    # Llamar a la función de evaluación
    res <- realidad_evaluar(drealidad_eval)

    # Guardar y luego imprimir el resultado del corte
    ganancias_ensemble_val <- rbind(ganancias_ensemble_val,
                                     list(envios, res$total))

    cat(sprintf("Corte %6d: Ganancia Total = %s\n",
                envios,
                format(res$total, big.mark = ".", decimal.mark = ",")))

    #  Limpiar la columna Predicted (Opcional, para el next loop)
    drealidad_eval[, Predicted := NULL]
}

corte_optimo <- ganancias_ensemble_val[which.max(ganancia_total)]

envios_optimos <- corte_optimo$envios
ganancia_maxima <- corte_optimo$ganancia_total

cat("\n------------------------------------------------------------\n")
cat("✅ CORTE ÓPTIMO EN VALIDACIÓN (202104):\n")
cat(sprintf("   Envíos Óptimos: %d\n", envios_optimos))
cat(sprintf("   Ganancia Máxima: %s\n", format(ganancia_maxima, big.mark = ".", decimal.mark = ",")))
cat("------------------------------------------------------------\n")


--- Curva de Ganancia del Super-Ensemble (202104) ---
Corte   6000: Ganancia Total = 346.400.000
Corte   6100: Ganancia Total = 349.200.000
Corte   6200: Ganancia Total = 350.400.000
Corte   6300: Ganancia Total = 3,5e+08
Corte   6400: Ganancia Total = 351.200.000
Corte   6500: Ganancia Total = 351.600.000
Corte   6600: Ganancia Total = 351.200.000
Corte   6700: Ganancia Total = 352.400.000
Corte   6800: Ganancia Total = 3,52e+08
Corte   6900: Ganancia Total = 352.400.000
Corte   7000: Ganancia Total = 352.800.000
Corte   7100: Ganancia Total = 354.800.000
Corte   7200: Ganancia Total = 355.200.000
Corte   7300: Ganancia Total = 3,54e+08
Corte   7400: Ganancia Total = 3,56e+08
Corte   7500: Ganancia Total = 3,54e+08
Corte   7600: Ganancia Total = 3,56e+08
Corte   7700: Ganancia Total = 358.800.000
Corte   7800: Ganancia Total = 3,6e+08
Corte   7900: Ganancia Total = 359.600.000
Corte   8000: Ganancia Total = 360.800.000
Corte   8100: Ganancia Total = 359.600.000
Corte   8200: Ganancia

In [67]:
require( "data.table" )

final0 <- fread("/content/prediccion_final_ensemble_4920.txt")[, .(numero_de_cliente, foto_mes, prob0 = prob)]
final1 <- fread("/content/prediccion_final_ensemble_4921.txt")[, .(numero_de_cliente, foto_mes, prob1 = prob)]
# final2 <- fread("/content/prediccion_final_ensemble_4922.txt")[, .(numero_de_cliente, foto_mes, prob2 = prob)]
final3 <- fread("/content/prediccion_final_ensemble_4923.txt")[, .(numero_de_cliente, foto_mes, prob3 = prob)]
final4 <- fread("/content/prediccion_final_ensemble_4924.txt")[, .(numero_de_cliente, foto_mes, prob4 = prob)]

# Definir los modelos A ENSAMBLAR y sus ganancias
# ganancias_a_ensamblar <- c(g0_max, g1_max, g3_max, g4_max)

# Calcular la suma total de las ganancias de los modelos seleccionados
suma_total_ganancia <- 1

# Calcular los pesos de ponderación
w0 <- 0.15
w1 <- 0.35
# w2 <- g2_max / suma_total_ganancia
w3 <- 0.35
w4 <- 0.15

# Lista de Data.Tables a ensamblar
tablas_a_ensamblar <- list(final0, final1, final3, final4)

# Combinar las probabilidades
ensamble <- Reduce(function(x, y) merge(x, y, by=c("numero_de_cliente","foto_mes")),
                   tablas_a_ensamblar)

# Calcular la probabilidad ponderada (Weighted Ensemble)
# Aplicamos la fórmula: prob_final = w0*prob0 + w2*prob2 + w4*prob4
ensamble[, prob_ponderada := (w0 * prob0) + (w1 * prob1) + (w3 * prob3) + (w4 * prob4)]

# Preparar el resultado final (ranking)
prediccion_final_ensemble_ponderado <- ensamble[, .(numero_de_cliente, foto_mes, prob = prob_ponderada)]

cat("Predicción ensemble ponderada (202106) calculada con éxito.\n")

Predicción ensemble ponderada (202106) calculada con éxito.


In [68]:
ensamble

numero_de_cliente,foto_mes,prob0,prob1,prob3,prob4,prob_ponderada
<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
249221323,202106,2.450380e-05,1.909201e-05,2.473674e-05,5.484496e-06,2.134990e-05
249227600,202106,2.796504e-05,2.724272e-05,2.615463e-05,6.466065e-06,2.680749e-05
249234235,202106,8.180193e-03,8.467029e-03,6.460368e-03,2.762191e-03,7.664364e-03
249244449,202106,2.124886e-04,1.134731e-04,7.351839e-05,8.856057e-05,9.749121e-05
249244739,202106,3.642991e-04,3.408341e-04,3.235388e-04,2.425146e-04,3.339160e-04
249255456,202106,4.823826e-05,4.994787e-05,6.437726e-05,1.324363e-05,5.571963e-05
249255747,202106,5.347564e-05,2.828821e-05,2.447637e-05,1.893763e-05,2.676347e-05
249257429,202106,1.885626e-04,2.091416e-04,2.197464e-04,1.222618e-04,2.133835e-04
249273979,202106,9.758283e-05,9.845503e-05,1.096245e-04,3.514114e-05,1.029228e-04


In [69]:
tb_prediccion <- prediccion_final_ensemble_ponderado[foto_mes == 202106, .(numero_de_cliente, foto_mes, prob)]
setorder(tb_prediccion, -prob)

In [70]:
# carpeta de salida en el espacio local de Colab
dir.create("/content/kaggle", showWarnings = FALSE)

envios <- 11300  # (asegurate de tener esta variable definida)

# marco los primeros
tb_prediccion[, Predicted := 0L]
tb_prediccion[1:envios, Predicted := 1L]

# ruta de salida absoluta
archivo_kaggle <- paste0("/content/kaggle/KA_ensemble_ponderado", envios, ".csv")

# guardo el CSV
fwrite(
  tb_prediccion[, .(numero_de_cliente, Predicted)],
  file = archivo_kaggle,
  sep = ","
)

cat("Archivo guardado en:", archivo_kaggle, "\n")

# si querés ver que realmente existe:
list.files("/content/kaggle")

Archivo guardado en: /content/kaggle/KA_ensemble_ponderado4921-23_12700.csv 


[1] "KA_ensemble_ponderado4920-21-22-23-24_10300.csv"
 [2] "KA_ensemble_ponderado4920-21-22-23-24_10500.csv"
 [3] "KA_ensemble_ponderado4920-21-23_11300.csv"      
 [4] "KA_ensemble_ponderado4920-21-23-24_10500.csv"   
 [5] "KA_ensemble_ponderado4920-21-23-24_11000.csv"   
 [6] "KA_ensemble_ponderado4920-21-23-24_11000(2).csv"
 [7] "KA_ensemble_ponderado4920-21-23-24_11300.csv"   
 [8] "KA_ensemble_ponderado4920-21-23-24_11300(2).csv"
 [9] "KA_ensemble_ponderado4920-21-23-24_11300(3).csv"
[10] "KA_ensemble_ponderado4921-23_11300.csv"         
[11] "KA_ensemble_ponderado4921-23_12500.csv"         
[12] "KA_ensemble_ponderado4921-23_12700.csv"         
[13] "KA_ensemble_ponderado4921-23_13000.csv"         
[14] "KA_ensemble4920-21-23-24_11800.csv"             
[15] "KA_ensemble4921-4923_11800.csv"